In [1]:
###IMPORTS

from __future__ import division

import collections
import array
import math

import rhinoinside
rhinoinside.load()
import Rhino

try:
    import Rhino.UI as rui
except ImportError as e:
    raise ImportError("Failed to import Rhino.\n{}".format(e))

IMPORT_EPW

In [2]:
import ladybug.epw as epw

_epw_file_path = "FRA_AC_Agen-La.Garenne.AP.075240_TMYx.epw"

epw_data = epw.EPW(_epw_file_path)
location = epw_data.location #######    
# dry_bulb_temperature = epw_data.dry_bulb_temperature
# dew_point_temperature = epw_data.dew_point_temperature
# relative_humidity = epw_data.relative_humidity
# wind_speed = epw_data.wind_speed
# wind_direction = epw_data.wind_direction
direct_normal_rad = epw_data.direct_normal_radiation #########
diffuse_horizontal_rad = epw_data.diffuse_horizontal_radiation #########
# global_horizontal_rad = epw_data.global_horizontal_radiation
# horizontal_infrared_rad = epw_data.horizontal_infrared_radiation_intensity
# direct_normal_ill = epw_data.direct_normal_illuminance
# diffuse_horizontal_ill = epw_data.diffuse_horizontal_illuminance
# global_horizontal_ill = epw_data.global_horizontal_illuminance
# total_sky_cover = epw_data.total_sky_cover
# barometric_pressure = epw_data.atmospheric_station_pressure
# model_year = epw_data.years
# g_temp = epw_data.monthly_ground_temperature
# ground_temperature = [g_temp[key] for key in sorted(g_temp.keys())]


SKYMATRIX

In [3]:
###IMPORTS SKY MATRIX
import math

try:
    from ladybug_geometry.geometry2d.pointvector import Vector2D
except ImportError as e:
    raise ImportError('\nFailed to import ladybug_geometry:\n\t{}'.format(e))

try:
    from ladybug_radiance.skymatrix import SkyMatrix
except ImportError as e:
    raise ImportError('\nFailed to import ladybug:\n\t{}'.format(e))

try:
    from ladybug_rhino.togeometry import to_vector2d
except ImportError as e:
    raise ImportError('\nFailed to import ladybug_rhino:\n\t{}'.format(e))

try:
    from lbt_recipes.version import check_radiance_date
except ImportError as e:
    raise ImportError('\nFailed to import honeybee_radiance:\n\t{}'.format(e))

# check the istalled Radiance date and get the path to the gemdaymtx executable
check_radiance_date()


In [4]:
###SKY MATRIX
# process and set defaults for all of the global inputs
north_ = 0
_location = location
_direct_rad = direct_normal_rad
_diffuse_rad = diffuse_horizontal_rad
_folder_ = "C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/cielMatrices"
 
if north_ is not None:  # process the north_
    try:
        north_ = math.degrees(
            to_vector2d(north_).angle_clockwise(Vector2D(0, 1)))
    except AttributeError:  # north angle instead of vector
        north_ = float(north_)

# create the sky matrix object
sky_mtx = SkyMatrix.from_components(
    _location, _direct_rad, _diffuse_rad, 
    #_hoys_, north_, high_density_, ground_r
    )
if _folder_:
    sky_mtx.folder = _folder_

print(sky_mtx.north)

0


#####
ORIENTATION

In [17]:
ORIENTATION_PARCELLE = 30 # [-180 ; 180] ; degrés ; orientation parcelle par rapport à l'axe Nord Sud
HAUTEUR = 3 # [0 ; 100] ; mètres ; hauter de l'axe de rotation, ou du point du PVP le plus bas.
ENTRAXE = 2.5 # [0 ; 10] ; mètres ; distance entre deux rang de panneaux, prises aux axes de rotation dans le cadre du tracking 
RAMPANT = 1.134 # [0 ; 10] mètres ; largeur des panneaux (dimension selon l'axe EST-OUEST)

# ptOri = Rhino.Geometry.Point3d(0,0,0)
# pt1 = Rhino.Geometry.Point3d(1,0,0)
# pt2 = Rhino.Geometry.Point3d(0,1,0)

# plan1 = Rhino.Geometry.Plane(ptOri,pt1,pt2)

ORIENTATION_PARCELLE_RAD = Rhino.RhinoMath.ToRadians(ORIENTATION_PARCELLE)
vecY = Rhino.Geometry.Vector3d.YAxis
vecY.Rotate(angleRadians = ORIENTATION_PARCELLE_RAD, rotationAxis = Rhino.Geometry.Vector3d.ZAxis)
print(vecY)

0,1,0
Origin=0,0,0 XAxis=1,0,0, YAxis=0,1,0, ZAxis=0,0,1
-0.5,0.866025403784439,0


TAUX DE COUVERTURE

In [42]:
NB_RANGS = 100/ENTRAXE
NB_PVP_RANG = 20

corn1 = Rhino.Geometry.Point3d(-50,-50,0)
corn2 = Rhino.Geometry.Point3d(-50,50,0)
corn3 = Rhino.Geometry.Point3d(50,50,0)
corn4 = Rhino.Geometry.Point3d(50,-50,0)

surf = Rhino.Geometry.NurbsSurface.CreateFromCorners(corn1,corn2,corn3,corn4)
brepSurf = surf.ToBrep()

pointGrid1 = Rhino.Geometry.Point3dGrid(int(NB_RANGS),NB_PVP_RANG)
a = surf.IsCone()
print(a)
print(pointGrid1.GetUserString)

False
<bound method 'GetUserString'>
